In [7]:
## 双子実験推定用ver1231
## 情報量em推定
## 観測分布・行動分布とも事象を条件付き分布に統一
## e-stepはQの更新，m-stepはQlogPの最大化によるsigma, thetaの同時推定
## 価値関数求解は構造化を外した

import pandas as pd
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

start_time = time.time()
print('start!')

start!


In [9]:
############################
####### reading data #######
############################

## これはtheta = [-1.5, 1, 0.8], sigma = [2, 25]で推定した結果→これに寄せる結果が出るかどうか
## これはBLEデータね！！！経路データではないよ！！！
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/generated_bledata_ver2.csv')

# search_folder = "/home/matsunaga/res2023/data/20230130_17/20230130_1745_18sec_end162021_under10_split/folder_10"
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_link.csv')
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_ble.csv')
df_demand = pd.read_csv('/Users/takahiromatsunaga/res2023/futago/df_demand.csv')
#file_list = list(os.listdir(search_folder))
L = len(df_link)
OD = len(df_demand)

In [10]:

############################
####### リンク接続行列 ####### ### 逆戻りと滞在ありの経路
############################

def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在
    for i in range(n):
        O = link_data.loc[i, 'o'] 
        D = link_data.loc[i, 'd'] 
        for j in range(n):
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = Imaking(df_link)

## スタートリンク25と吸収リンク26だけ操作
# リンク25(index24)へはどこからもいかない（スタート→スタートもない）
I[:, 24] = 0

# リンク26(index25)からは26にしかいかない
I[25, :] = 0
I[25, 25] = 1

#########################################
####### 吸収リンク＆タイムステップ設定 #######
#########################################

# 吸収はリンク26（index25, 仮想ノード17(100, 100, 0)に接続）のみ
ddata = [26]
D = len(ddata)

## 観測パラメータ
#b = np.array((2, 25)) # N = b[0], sigma = b[1]
max_rssi = -40 ## これは固定して，この値は超えないように観測分布も与えるようにする

# 最短でT=7でスタートリンクから吸収リンクまで到達．
T = 10 # 試しに10で

##########################
###### 時空間プリズム ######
##########################
def TSNW(ODlist): # df_demandを与えれば良い
    Ilist = []

    # 起点Oからの到達可能性指示変数io
    Itlist = np.zeros((T, L, L)) ## これは各状態に対して与えるので遷移回数T-1よりひとつ多くてT成分考える（timestep数がTなら状態数はT）
    II = np.copy(I)
    Itlist[0, :, :] = np.eye(L)

    for ts in range(1, T): 
        Itlist[ts, :, :] = II 
        II = np.dot(II, I)
        II = np.where(II > 0, 1, 0) 

    # 終点Dからの到達可能性指示変数id ## これもIlist同様の考え方
    Ittlist = np.zeros((T, L, L))
    for ts in range(T): 
        Ittlist[ts, :, :] = np.transpose(Itlist[T - ts - 1, :, :]) 

    for od in range(len(ODlist)):   

        # OD確定読み込み
        ao = ODlist.loc[od, 'o'] - 1  # index
        aabs = ODlist.loc[od, 'd'] - 1  # index
        #aabs = ODlist.loc[od, 'abs'] - 1
        #userid = df_demand.loc[od, 'userid']

        Id = np.zeros((T-1, L, L)) # Id[t]は時刻tの時の利用可能遷移k→aを示す．最後のt=T-1のとき不要なので要素数はt=0~T-2のT-1個

        for ts in range(T-1): # Tまでに吸収されるのでdはaabs．遷移回数はT-1なのでこれでOK
            if ts == 0:
                Id[ts, ao, :] = I[ao, :] 
                continue
            
            alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, aabs, :])[0] ## ts=1（つまり第二回目の状態）のとき3番目の状態を見ている→OK
            ## Itlistの長さはT．最後はT-2で，そのときT-2+1=T-1, 
            for a in alist:
                if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, aabs, a]: # always True
                    klist = np.where(I[:, a] == 1)[0]

                    for k in klist:
                        if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                            Id[ts, k, a] = 1 

        Ilist.append(Id)
    
    return Ilist


###########################
######### 諸々の準備 ######## 
########################### 

###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['z'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['z'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)
    link_loc_array.append(loc_tuple) 


###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'bleid']
    x_j = df_ble[df_ble['bleid'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['bleid'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['bleid'] == beacon_id]['z'].iloc[0]
    
    beacon_loc = np.array([x_j, y_j, z_j])
    beacon_loc_array.append(beacon_loc)


###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)

    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算
    distance = np.linalg.norm(x - mid_point)
    
    return distance


###### d_arrayの用意 ###### # 各リンクの中点とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))
for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'z']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離



In [11]:

#######################
####### 観測モデル ######　## ここは，あくまで観測モデルの分布を与えているに過ぎないので共通でいい気がする．尤度関数の部分か．
#######################  ## 経路を確定させるから大山羽藤2017と同じ，Tmatrixは条件付き分布として与える（Pと対象事象を揃えるため）

def qjx(rssi, x_loc, j, b):               # bは観測誤差分散のパラメータ．x_loc（x, y, z)にいたときビーコンjでrssiの観測が得られる確率
    j_loc = beacon_loc_array[j-1]                   # id=jのビーコンの位置座標
    d_est = 10 ** ((max_rssi - rssi)/(10*b[0]))     # 減衰パラメタb[0] rssi0→-40にしてみる
    d = np.linalg.norm(x_loc - j_loc) 
    f = norm.pdf(d, d_est, b[1])                    # 誤差分散b[1]
    omega = (100 + rssi) / 10 

    return omega * f


# -----****------Qを与える．Qはthetaとsigmaで決まる分布，同時推定なのでthetaとsigmaが両方xに入る-----****-----
def Mmodel(x, b):

    Pall = newPall(x)
    count = 0 
    
    SLlist = []
    TLlist = []

    grouped = df_data.groupby('d') # odで場合わけ．今回はdで分けることにする．不要だけど拡張を見据えて．
    df_list = [group.reset_index(drop = True) for name, group in grouped]

    for od in range(OD):
        TLodlist = []
        dfod = df_list[od] # 今回はこれしかないのでここで全データが読み込まれる
        Id = Ilist[od] # プリズムもodごとに1個に決まるのでみんな共通
        ao = int(dfod.loc[0, 'o']) - 1 # index 配分結果にo列与えてなかったわ
        #ao = dfod.loc[0, 'k'] - 1 
        aabs = int(dfod.loc[0, 'd']) - 1 # index

        grouped2 = dfod.groupby('userid')
        df_list2 = [group.reset_index(drop = True) for name, group in grouped2]

        for i in range(len(df_list2)): # 個人iに対して
            dfi = df_list2[i]
            ad = int(dfi.loc[len(dfi)-1, 'd']) - 1
                
            grouped3 = dfi.groupby('timestep')
            dfi_list = [group.reset_index(drop=True) for name, group in grouped3] ## 個人のデータをtimestepごとに分けている

            Ti = len(dfi_list)  ## ao入れるなら+1する必要(Ti<=9)のはず                    # 個人iの観測限界．時刻Tiでlast_linkに到達し，時刻Ti+1で吸収される

            #### 観測プリズム用意 #### odを与えたらプリズム制約．時刻TiつまりTi-1回の遷移でaoからadまで行く
            Idm = np.zeros((Ti-1, L, L)) ## 1成分目の値が遷移回数．Tiにするならabsまで行くことを仮定している．そのときItimlistは状態数＝時間数なのでTi+1で良い
            Itmlist = np.zeros((Ti, L, L))
            IIm = np.copy(I)
            Itmlist[0, :, :] = np.eye(L)

            for tt in range(1, Ti): 
                Itmlist[tt, :, :] = IIm 
                IIm = np.dot(IIm, I) 
                IIm = np.where(IIm > 0, 1, 0)

            Ittmlist = np.zeros((Ti, L, L))
            for tt in range(Ti): 
                Ittmlist[tt, :, :] = np.transpose(Itmlist[(Ti) - tt - 1, :, :])

            for tt in range(Ti-1):
                if tt == 0:
                    #print(ao)
                    Idm[tt, ao, :] = I[ao, :] 
                    #print('iketeruyo')
                    continue

                alist = np.where(Itmlist[tt + 1, ao, :] == Ittmlist[tt + 1, aabs, :])[0] # ここadじゃなくてaabsが適切では????
                for a in alist:
                    if Itmlist[tt + 1, ao, a] == Ittmlist[tt + 1, aabs, a]: # ここadじゃなくてaabsが適切では????
                        klist = np.where(I[:, a] == 1)[0] 

                        for k in klist:
                            if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                                Idm[tt, k, a] = 1 ### 時刻tにおいて，a(t)→a(t+1)への遷移がOKなところが1になっている                            

            #SLmatrix = np.zeros((T, L)) # status_likelihood 状態数は遷移数+1
            TLmatrix = np.zeros((T-1, L, L)) # trans likelihood
            pmmatrix = np.zeros((T-1, L, L))  # p(m|a)のmatrixのはず
            pmstmatrix = np.zeros((T-1, L, L)) # 同時発生尤度のマトリックス
            #SLmatrix[0, ao] = 1 # t = 0ではaoに確定的に存在するとする ####### ←ここがまずよくない．
            klist = [ao] # klistの初期値（oは確定）
            
            for t in range(1, T): #### t=tでは，k=a(t-1)からa=a(t)への遷移を考える．TLmatrixは遷移数＝T-1個しかないのでindexは0 ~ t-2

                #######################################
                ######### 1. 遷移発生尤度を計算 ########## # t=0はそもそもループから除かれている
                #######################################

                if t <= Ti - 1: ## この時間は遷移している．Ti<=t<=T-1では吸収されている

                    dfit = dfi_list[t] # Ti=len(dfi_list)+1としてしまっているのがよくない!!

                    ts = int(dfit.loc[0, 'timestep'])

                    it_grouped = dfit.groupby('bleid')
                    dfit_list = [it_group.reset_index(drop=True) for name, it_group in it_grouped]  

                    #slsum = 0
                    #nlist = np.argsort(SLmatrix[t-1])[::-1]  # tsにおける状態尤度が大きい順に並び替えたインデックスリスト
                    ## t=1のときSL[0]を参照するがaoしかないのでそのままaoを見ることになる．t=2のときSL[1]を参照，tのときt-1を参照
                    ## ループは1からまでにしてt-1における尤度が大きいリンクに対して??実装?
                    ## t-1で尤度大きいリンクをkとしてtのリンクをaとして扱った方が実態に近い
                    #klist = [] # つまり時刻t-1で状態尤度が高い上位のリンクをklistに入れている
                    #for i in range(L):
                    #    slsum += SLmatrix[t-1, nlist[i]]
                    #    klist.append(nlist[i])
                    #    if slsum >= 0.9:
                    #    break 
                    #klist3 = [i+1 for i in klist]
                    #print(f'時刻{t-1}での状態発生尤度の高かったリンクは{klist3}') ##

                    for k in klist: 

                        for a in range(L): ## tのリンクはこっち

                            if Idm[t-1, k, a] == 0: ### t-1時点でk→aが利用可能か否かを見る ###### 
                                TLmatrix[t-1, k, a] = 0 ### 多分これでいいのか？## TLmatrix[0]はao→a1が入っている
                                continue

                            a_loc = link_loc_array[a]
                            a_o = a_loc[0]
                            a_d = a_loc[1]

                            ## 位置については一旦保留→aの上に確率的に存在するという仮定が望ましいが一旦リンクの中点にいるとする
                            x_loc = (a_o + a_d) / 2

                            pm = 1

                            ######## 全観測データを使う場合 ##########
                           
                            for j in range(len(dfit)): ### ここでようやく各観測ログにアプローチ
                                ### rssi小さい観測がpmの極小化を招いている説もある，積をとっているので．
                                ### でも相対評価で尤度最大化しようとしているならそこまで気にならないはずだが
                                j_id = int(dfit.loc[j, 'bleid'])
                                j_rssi = dfit.loc[j, 'rssi']

                                pm *= qjx(j_rssi, x_loc, j_id, b) # 観測方程式
                            
                            """
                            ######### 各ビーコンで最大強度の観測データのみ使う場合 #########
                            ###### ビーコンごとにgroupbyしてその中でrssiが最大のログに対してpmを計算する
                            for j in range(len(dfit_list)):
                                dfitj = dfit_list[j]
                                j_id = int(dfitj.loc[0, 'bleid'])
                                # j_max_rssi = max_jt[j_id - 1, ts - 1] 
                                # dfitの中で，RSSI列が最大の要素のRSSIの値を取得
                                j_rssi = dfitj['rssi'].max()

                                pm *= qjx(j_rssi, x_loc, j_id, b) # 観測方程式
                            """
                            pmmatrix[t-1, k, a] = pm # 時刻tの観測尤度はリンクaの観測確率ね→ただしindexの関係でt-1に入れる．つまりt=1にat-1=a0=k→at=a1=aの観測尤度がpm[0]に入ってる
                            #print(f'時刻{t}でリンクa(t)つまり{a+1}の観測尤度{pm}')

                    # 遷移尤度のnume pmst 時刻tにおいてk(t-1)→a(t)への遷移が観測された確率がt-1に入ってる
                    # 時刻tにおいてk(t-1)→a(t)への遷移確率がPall[t-1]に入っている=時刻t-1にk(t-1)→a(t)に遷移する確率がPall[t-1]ということ
                    # つまり時刻tにおけるat→at+1への遷移確率はPall[t]に入っているということか．で，aT-1→aTへの遷移確率まで入っているがaT-2→aT-1までしか使わない
                    pmstmatrix[t-1, :, :] = pmmatrix[t-1, :, :] * Pall[od, t-1, :, :] # 観測確率pm*遷移確率（正規化前の観測尤度）

                    # 正規化
                    deno = np.sum(pmstmatrix[t-1, :, :], axis=1) # np.sum(pmstmatrix[t-1], axis=1)になってた
                    deno = (deno == 0) * 1 + (deno != 0) * deno ## 観測尤度の分母
                    
                    deno_rep = np.tile(deno, L).reshape(L, L).T ##### これがないと変になった

                    pmstmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] / deno_rep ## これが観測尤度

                    #sl = SLmatrix[t-1, :] # このままでは行ベクトル
                    #sl_col = sl[:, None] #### ここがRと文法異なる部分

                    TLmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] # * sl_col # 同時発生尤度を計算してる（SLmatrixはindexが1増えてるので，同じt-1でもひとつ前の段階を指名している
                    # 時刻tにk(a_t-1)からa(a_t)への遷移が発生する尤度

                    #### klist更新
                    klist = []
                    TLt = TLmatrix[t-1, :, :]
                    # 方法1: any() を使って非零の要素を持つ列を見つける
                    non_zero_columns = np.any(TLt != 0, axis=0)
                    klist = np.where(non_zero_columns)[0]

                    # 方法2: nonzero() を使って非零の要素がある列のインデックスを取得する
                    non_zero_indices = np.nonzero(np.any(TLt != 0, axis=0))[0]
                    klist = non_zero_indices.tolist()

                elif t == Ti:
                    TLmatrix[t-1, ad, aabs] = 1 ### それ以外はそのまま→0のまま
                
                else:
                    TLmatrix[t-1, aabs, aabs] = 1 ### それ以外はそのまま→0のまま

                ########################################
                ########## 2. 状態発生尤度を定義 ##########
                ########################################
                """
                if t <= Ti - 2: # 観測プリズム（ao, ad, Tiから決定）Idm内の状態しか取り得ない．k(a_t-1)からa(a_t)への遷移がOKのときIdmt[t-1]が1

                    dfit = dfi_list[t] # t<len(dfit)=Tiにおいて指定可能
                    ts = int(dfit.loc[0, 'timestep'])

                    for i in range(L): ## iは遷移先リンク，つまりa(t)の候補ということになる

                        klist = np.where(Idm[t-1, :, i] == 1)[0] ## 前時刻参照．時刻tにおいてリンクi=a(t)に遷移可能な前リンクk=a(t-1)を探し，klistに入れている．この時，構造化NWと観測情報からリンク候補はかなり絞られている

                        if len(klist) == 0:
                            continue

                        kTL = 0
                        for k in klist: # このiに移動できる元リンク（t-1時点でのリンクa(t-1)）
                            kTL += TLmatrix[t-1, k, i] ## 時刻t-1にk=a(t-1)からi = a(t)への遷移発生尤度をkTLに足していき，これがtでのiの存在尤度になる（それはそう！！）
                            #print(f'状態遷移 k = a(t-1) = {k+1} → a = a(t) = {i+1}（全部リンクid）の遷移発生尤度は{kTL}')

                        SLmatrix[t, i] = kTL ### SLmatrixはindex t に入れる（SL[0]を事前に定義済みなので）

                if t == Ti - 1: # この時刻で最終リンクに確定的に滞在
                    #print(f'時刻{t}で最後のリンク')
                    SLmatrix[t, ad] = 1

                elif t >= Ti: # この時刻以降は確定的に吸収に滞在
                    #print(f'時刻{t}で吸収リンク')             
                    SLmatrix[t, aabs] = 1
                """

            #SLlist.append(SLmatrix)
            TLodlist.append(TLmatrix)
        TLlist.append(TLodlist)

    return(TLlist)



In [12]:

###########################
####### 経路選択モデル ######
###########################

###### 即時効用行列 ###### # 混雑内生性考えないならT次元は圧縮可能だが一応→時不変だし，経路長も一緒だし経路パラメタしか入れてないので全リンクの効用が等しくなるよ．
def Mset(x): 
    #Probs_T = Probs.T # Probsを読み込む（内生性バージョン）
    inst = np.zeros((T-1, L, L))
    for t in range(T-1):
        inst_t = np.exp(df_link['cc'] * x[0] + df_link['dc'] * x[1]) # + df_link['length'] * x[2])    ### + Probs_T[:, t] * x[1])    # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
        inst_t = pd.concat([inst_t]*L, axis=1) 
        inst_t = inst_t.T
        inst_t_numpy = inst_t.values # DataFrameをNumPy配列に変換
        inst[t, :, :] = inst_t_numpy
    return inst


###### リンク選択確率計算 ######
def newPall(x):
    Pall = np.zeros((OD, T-1, L, L)) # 今回Tを状態の数としているので遷移数はT-1になる
    beta = x[-1]

    for od in range(OD): 
        Id = Ilist[od]  
        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] 
            M[ts, :, :] = Mts
        
        ##### 価値関数 ##### # ここで価値関数求解（newVに相当），価値関数別で計算するならここにVを外から読み込むことになる
        z = np.ones((T, L))
        for t in range(T-1, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        ## 価値関数構造化しなくてもちゃんと収束してるかの確認（普通にしてるだろ？？）
        # print(f'今zは{np.sum(np.abs(z))}') ## してると見えた

        ##### 選択確率行列 ##### 
        for t in range(T-1):
            for k in range(L):
                for a in range(L):
                    if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除（logzero回避）
                        continue # ここがbreakになってたのが癌だった
            
                    Pall[od, t, k, a] += np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])) 
    return Pall 


#### 時間/リンク別期待人数行列 ###
def ec_set(x):
    ec_matrix = np.zeros((T-1, L))
    beta = x[-1]
    Pall = newPall(x)

    for od in range(OD):
        ao = od_list.loc[od, 'o'] - 1
        Pi = Pall[od, :, :, :]
        LCP = np.zeros((T, L))
        
        for t in range(T-1):
            if t == 0:
                LCP[0, ao] += 1
                ec_matrix[0, :] += LCP[0, :]
            else:
                LCP[t, :] = np.dot(LCP[t-1, :], Pi[t, :, :])
                ec_matrix[t, :] += LCP[t, :]
    
    return ec_matrix




In [13]:

# これどこでsigma推定するのだ→ひとまず固定
### 最尤法の際の逐次リンク確定の関数 ## 個人ごとにリンク系列を出す
def link_sequence(x, b): # TLmatrixを個人ごとに用意して，各時刻で最大のリンクを取得→接続するリンクから，また次の最大リンクを取得→df_sequenceを作成→loglikelihoodでdf_sequenceを読み込んで真の分布として最尤推定
    TLlist = Mmodel(x, b) # x_k, b_kで固定した場合の観測分布から計算する

    ## userid, timestep, maxlink, d(=ad+1)
    res_all = np.zeros(5) # 結果格納用
    
    for od in range(OD):
        TLodlist = TLlist[od]
        ao = df_demand.loc[od, 'o'] - 1 # 最初のリンクのindex(24のはず)
        ad = df_demand.loc[od, 'd'] - 1

        for i in range(len(TLodlist)):
            TLmatrix = TLodlist[i] # 個人iの状態遷移発生尤度．観測限界以降は26が1，それ以外が0になっている
            userid = i 
            k = ao # 初期値はao．前時刻で尤度が最大だったリンクをkに入れてtループ内で毎回更新
            a = ao # 初期値はao，t=0ですぐ更新する

            for t in range(T-1): 
                max_link_idx = np.argmax(TLmatrix[t, k, :]) # TLmatrix[0]はao→a1が入っている
                max_link_id = max_link_idx + 1 # idに
                a = max_link_idx # TLmatrixの中身が0とかじゃなくてちゃんと入っていれば，0になって破綻するとかにはならないと思う

                # 個人の結果は出たので入れる
                res_it = np.array((userid, t, k, max_link_id, ad+1))
                res_all = np.vstack((res_all, res_it))
                k = a 
    
    res_all = np.delete(res_all, 0, axis = 0) # 冒頭の0の行を削除
    df_res_all = pd.DataFrame(res_all, columns = ['userid', 'timestep', 'k', 'a', 'd'])

    print(df_res_all)

    return df_res_all

In [14]:

######################
####### 最尤推定 ######
######################

# m-step（リンク固定して最尤推定）
def loglikelihood(x): # TLmatrixとdf_res_all はx_k, b_kで固定しておき，選択確率パラメタのxのみ動かす
    LL = 0
    Pt = newPall(x)
    
    ## ちょっと納得いかないけど，，
    TLlist = Mmodel(x0, b0)
    df_res_all = link_sequence(x0, b0) # e-stepでリンク系列は確定させる．→外に出したほうがいい？？

    for od in range(OD):
        TLodlist = TLlist[od] # TLlistを所与とする
        grouped = df_res_all.groupby('userid')
        df_res_list = [group.reset_index(drop = True) for name, group in grouped]

        for i in range(len(TLodlist)): # i: userid 
            TLmatrix = TLodlist[i]
            dfi = df_res_list[i] # i番目の人のリンク系列

            for t in range(T-1):
                k = int(dfi.loc[t, 'k']) - 1
                a = int(dfi.loc[t, 'a']) - 1
                pka = Pt[od, t, k, a]
                pka = (pka == 0) * 1 + (pka != 0) * pka 

                LL += np.log(pka)

    return -LL


In [ ]:
x_init = np.zeros(3)

x0 = x_init
b0 = np.array((2, 5))
Ilist = TSNW(df_demand)